In [1]:
from langchain_community.document_loaders import PyPDFLoader

In [8]:
loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader("/home/yzhao/projects/RD/LangChain/document/A Complete Guide to the Futures Market Technical Analysis, Trading Systems, Fundamental Analysis, Options, Spreads, and... (Jack D. Schwager) (Z-Library).pdf"),
]
loader = loaders[0]
pages = loader.load_and_split()

In [10]:
print(len(pages))
page = pages[500]
# print(dir(page))
print(page.page_content)

690
508
A Complete Guide to the Futures mArkettabLe 35.5d  profit/Loss Matrix for Long puts with Different Strike prices
Dollar amount of   premium paid
$1,350 
put$1,300 
put$1,250 
put$1,200 
put$1,150 
put$1,100 
put$1,050 
put
$15,410 $10,870 $6,870 $3,870 $1,990 $1,010 $510
position profit/Loss at expiration
Futures price at 
expiration ($/oz)Short Futures 
at $1,200In-the-Money at-the-Money Out-of-the-Money
$1,350 
put$1,300 
puta$1,250 
put$1,200 
puta$1,150 
put$1,100 
puta$1,050 
put
1,000 $20,000 $19,590 $19,130 $18,130 $16,130 $13,010 $8,990 $4,490
1,050 $15,000 $14,590 $14,130 $13,130 $11,130 $8,010 $3,990 –$510
1,100 $10,000 $9,590 $9,130 $8,130 $6,130 $3,010 –$1,010 –$510
1,150 $5,000 $4,590 $4,130 $3,130 $1,130 –$1,990 –$1,010 –$510
1,200 $0 –$410 –$870 –$1,870 –$3,870 –$1,990 –$1,010 –$510
1,250 –$5,000 –$5,410 –$5,870 –$6,870 –$3,870 –$1,990 –$1,010 –$510
1,300 –$10,000 –$10,410 –$10,870 –$6,870 –$3,870 –$1,990 –$1,010 –$510
1,350 –$15,000 –$15,410 –$10,870 –$6,870 –$3

In [11]:
page.metadata

{'source': '/home/yzhao/projects/RD/LangChain/document/A Complete Guide to the Futures Market Technical Analysis, Trading Systems, Fundamental Analysis, Options, Spreads, and... (Jack D. Schwager) (Z-Library).pdf',
 'page': 525}

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [13]:
docs = []
for loader in loaders:
    docs.extend(loader.load())
splits = text_splitter.split_documents(docs)
len(splits)

1337

解析文件 使用本地模型，huggingface的模型或者openai的都可以

In [14]:
from langchain.embeddings import HuggingFaceEmbeddings

# Path to your local model
local_model_path = "/home/yzhao/projects/RD/LangChain/model/mistral7b"
# local_model_path = "/home/yzhao/projects/RD/LangChain/model/bert-base-uncased"
# Using open source model

# Instantiate the HuggingFaceEmbeddings class
model_name = local_model_path
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)



/home/yzhao/anaconda3/envs/LLM/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: Path /home/elinharespontes/mistral/rene-chat/models/Mistral-7B-Instruct-v0.2 not found

In [10]:
from transformers import AutoTokenizer

# Load the tokenizer for the HuggingFace model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define a padding token for the tokenizer
tokenizer.pad_token = '[PAD]'

Test the embemding works...

In [11]:
sentence1 = "I love dogs"
sentence2 = "I like dogs"
sentence3 = "I hate dogs"

In [12]:
embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

: 

In [ ]:
import numpy as np

In [ ]:
np.dot(embedding1, embedding2)

In [ ]:
np.dot(embedding1, embedding3)

In [ ]:
np.dot(embedding2, embedding3)

Vertorization

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
persist_directory = 'document/chroma/'
!rm -rf ./docs/chroma  # remove old database files if any

In [ ]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [ ]:
print(vectordb._collection.count())

In [ ]:
question = "what is double bottom"

In [ ]:
docs = vectordb.similarity_search(question,k=3)

In [ ]:
len(docs)

In [ ]:
docs[0].page_content

In [ ]:
vectordb.persist()

In [ ]:
#Add more text to the database
texts = [
    """The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).""",
    """A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.""",
    """A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.""",
]

In [ ]:
smalldb = Chroma.from_texts(texts, embedding=embedding)

In [ ]:
question = "Tell me about all-white mushrooms with large fruiting bodies"

In [ ]:
smalldb.similarity_search(question, k=2)

In [ ]:
smalldb.similarity_search(question, k=2)

In [ ]:
question = "what did they say about matlab?"
docs_ss = vectordb.similarity_search(question,k=3)

In [ ]:
docs_ss[0].page_content[:100]

In [ ]:
docs_ss[1].page_content[:100]